In [2]:
import pandas as pd
import import_ipynb
from data_cleaning import df_locations

In [12]:
df_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id             1557740 non-null  int64  
 1   order_id       1557740 non-null  int64  
 2   driver_id      1557740 non-null  int64  
 3   driver_action  1557740 non-null  object 
 4   lat            1557740 non-null  float64
 5   lng            1557740 non-null  float64
 6   created_at     0 non-null        float64
 7   updated_at     0 non-null        float64
dtypes: float64(4), int64(3), object(1)
memory usage: 95.1+ MB


In [4]:
df_locations['driver_id'].nunique()

1213

In [9]:
df_accepted = df_locations[df_locations['driver_action']=='accepted']
df_accepted

,id,order_id,driver_id,driver_action,lat,lng,created_at,updated_at
0,1,392001,243828,accepted,6.602207,3.270465,NaN,NaN
10,11,392005,245597,accepted,6.549147,3.392184,NaN,NaN
61,62,392009,245600,accepted,6.644829,3.289328,NaN,NaN
128,129,392013,243892,accepted,6.435331,3.424317,NaN,NaN
141,142,392014,243781,accepted,6.498221,3.360042,NaN,NaN
...,...,...,...,...,...,...,...,...
1557594,1557595,489898,246615,accepted,6.443810,3.464208,NaN,NaN
1557614,1557615,490008,247877,accepted,6.448554,3.574380,NaN,NaN
1557670,1557671,497721,243414,accepted,6.498164,3.359964,NaN,NaN
1557673,1557674,510268,243532,accepted,6.462251,3.391023,NaN,NaN


Accepted = 25,903 from  1,557,740 

In [11]:
df_accepted = df_locations[df_locations['driver_action']!='accepted']
df_accepted

,id,order_id,driver_id,driver_action,lat,lng,created_at,updated_at
1,2,392001,243588,rejected,6.592097,3.287445,NaN,NaN
2,3,392001,243830,rejected,6.596133,3.281784,NaN,NaN
3,4,392001,243539,rejected,6.596142,3.280526,NaN,NaN
4,5,392001,171653,rejected,6.609232,3.288800,NaN,NaN
5,6,392001,245662,rejected,6.593095,3.287759,NaN,NaN
...,...,...,...,...,...,...,...,...
1557735,1557736,517948,243774,rejected,6.469036,3.566877,NaN,NaN
1557736,1557737,517948,245447,rejected,6.425431,3.527358,NaN,NaN
1557737,1557738,517948,239866,rejected,6.440013,3.525378,NaN,NaN
1557738,1557739,517948,243774,rejected,6.469036,3.566877,NaN,NaN


Rejected = 1,531,837 from =  1,557,740 